In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

In [ ]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))

nb_train_samples = 18047
nb_val_samples = 4377
image_size = (224,224)
batch_size = 128
class_num = 10

In [ ]:
resNet50_base_model = ResNet50(include_top=False, weights='imagenet',pooling='avg')

x = resNet50_base_model.output
x = Dropout(0.5)(x)
x = Dense(10,activation='softmax')(x)

resNet50_model = Model(inputs=resNet50_base_model.input,outputs=x)

resNet50_model.summary()
for i,layer in enumerate(resNet50_model.layers):
    print(i,layer.trainable,layer.name)

In [ ]:
train_datagen = image.ImageDataGenerator(rescale=1./255,
                                         shear_range=0.2,
                                         rotation_range=10,
# #                                          height_shift_range=0.1,
#                                          width_shift_range=0.1,
                                         channel_shift_range=5,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(rescale=1./255,
                                             preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)


In [ ]:
resNet50_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                      #optimizer=optimizers.Adam(),
                    optimizer='rmsprop',
                    metrics=['accuracy'])

# fine-tune the model
resNet50_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/resnet50_fine_tune'),
                                   CSVLogger('resnet50_fine_tune'),
                                   EarlyStopping(patience=3)])